# Using Shapeless to derive instances

In [1]:
import $ivy.`com.chuusai::shapeless:2.3.3`, shapeless._

import $ivy.$                             , shapeless._

In [2]:
trait Ordering[A] {
  def compare(left: A, right: A): Int
}

object Ordering {
  def apply[A](implicit ev: Ordering[A]): Ordering[A] = ev
}

implicit val compareInt: Ordering[Int] = (left: Int, right: Int) => left - right

implicit val compareString: Ordering[String] = (left: String, right: String) => {
  if(left.length == right.length) {
    left.zip(right).zipWithIndex.find { case ((l, r), _) => l != r }.map(_._2 + 1).getOrElse(0)
  } else math.min(left.length, right.length) + 1
}

defined trait Ordering
defined object Ordering
compareInt: Ordering[Int] = ammonite.$sess.cmd1$Helper$$anonfun$1@36dd0f20
compareString: Ordering[String] = ammonite.$sess.cmd1$Helper$$anonfun$2@180aee83

# Base Case

In [3]:
implicit val compareHNil: Ordering[HNil] = (_: HNil, _: HNil) => 0

compareHNil: Ordering[HNil] = ammonite.$sess.cmd2$Helper$$anonfun$1@5f75e794

# Implicit Step

In [4]:
implicit def compareHList[Head, Tail <: HList](implicit
                                               headCompare: Ordering[Head],
                                               tailCompare: Ordering[Tail]) = new Ordering[Head :: Tail] {
  override def compare(left: Head :: Tail, right: Head :: Tail) = {
    val leftC = headCompare.compare(left.head, right.head)
    lazy val rightC = tailCompare.compare(left.tail, right.tail)

    if(leftC == 0) rightC else leftC
  }
}

defined function compareHList

## Implicit Case Class Derivation

In [5]:
implicit def compareCaseClasses[CC, HL <: HList](implicit gen: Generic.Aux[CC, HL],
                                           hListC: Ordering[HL]): Ordering[CC] = new Ordering[CC] {
  override def compare(left: CC, right: CC): Int = {
    val lHL = gen.to(left)
    val rHl = gen.to(right)

    hListC.compare(lHL, rHl)
  }
}

defined function compareCaseClasses

In [6]:
case class User(name: String, age: Int)

val transformer = Generic[User]

transformer.from(HList("name", 42))
transformer.to(User("name", 42))

Ordering[String :: Int :: HNil].compare(HList("name", 42), HList("name", 43))

Ordering[Int].compare(1, 1)
Ordering[String].compare("a", "a")
Ordering[User].compare(User("name", 42), User("", 42))

defined class User
transformer: Generic[User]{type Repr = String :: Int :: shapeless.HNil} = ammonite.$sess.cmd5$Helper$anon$macro$3$1@66980d0a
res5_2: User = User("name", 42)
res5_3: transformer.Repr = "name" :: 42 :: HNil
res5_4: Int = -1
res5_5: Int = 0
res5_6: Int = 0
res5_7: Int = 1

In [7]:
case class Other(a: Long, b: User)

val transformer2 = Generic[Other]

//Ordering[Other].compare(Other(1, User("a", 1)), Other(1, User("a", 2))) // Will not work becasuse we don't have Ordering instance of `Long`

defined class Other
transformer2: Generic[Other]{type Repr = Long :: cmd6.this.cmd5.User :: shapeless.HNil} = ammonite.$sess.cmd6$Helper$anon$macro$3$1@68cfcce9

## Case Class Transformation

In [8]:
trait Transform[A, B] {
  def from(v: A): B
  def to(v: B): A
}

implicit def transformCaseClasses[CC1, CC2, HL <: HList](implicit gen1: Generic.Aux[CC1, HL],
                                                         gen2: Generic.Aux[CC2, HL]) = new Transform[CC1, CC2] {
  override def from(v: CC1) = {
    gen2.from(gen1.to(v))
  }

  override def to(v: CC2) = {
    gen1.from(gen2.to(v))
  }
}

case class Person(name: String, age: Int)

defined trait Transform
defined function transformCaseClasses
defined class Person

In [9]:
implicitly[Transform[User,Person]].from(User("A", 1))
implicitly[Transform[User,Person]].to(Person("A", 1)) 

res8_0: Person = Person("A", 1)
res8_1: User = User("A", 1)